In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import re

max_price = 800000
min_area = 50
pages = 2

def get_links(max_price, min_area, pages):

    try:

        for page in range(pages):
            url = 'https://loft.com.br/venda/apartamentos/rj/rio-de-janeiro/?bairros=flamengo_rio-de-janeiro_rj%7Ebotafogo_rio-de-janeiro_rj&precoMax='+str(max_price)+'&pagina='+str(page)+ '&areaMin='+str(min_area)

            # Request the page
            driver = webdriver.Chrome()
            driver.get(url)
            # Get scripts with data
            apts_data = driver.find_elements(By.XPATH, '//script[@type="application/ld+json"]')

            #collect links
            links = []
            for apt_data in apts_data:
                data = json.loads(apt_data.get_attribute('innerHTML'))
                link = data['url']
                links.append(link)
            
            print(len(links))

    except Exception as e:
        print(e)
    
    finally:
        driver.quit()

    return links


def get_apt_data(link):
    
    try:
        driver = webdriver.Chrome()
        driver.get(link)

        # Create dict to store data
        apt_data = {}

        # Costs data (price, condo, iptu)
        costs = driver.find_element(By.CLASS_NAME, 'jss487').text.split('\n')
        print(costs)
        # Substring keys to find info in costs
        costs_key_mapping = {'price': 'Valor à vista', 'cond.': 'Condomínio', 'iptu': 'IPTU'}
        # Fill apt_data with costs info
        for key, value in costs_key_mapping.items():
            value_index = costs.index(value)
            print(key, value, value_index)
            apt_data[key] = re.sub(r'\D', '', costs[value_index + 1]) if value_index else None 

        # Basic info (area, bedrooms, bathrooms, garage, suites, subway_distance)
        basic_info = driver.find_element(By.XPATH, '//div[@class="MuiGrid-root MuiGrid-container"]').text.split('\n')
        # Substring keys to find info in basic_info
        basic_info_key_mapping = {
            'area': ' m²',
            'bedrooms': 'quarto',
            'bathrooms': 'banheiro',
            'garage': 'vaga',
            'suites': 'suíte',
            'subway_distance': 'estação',
        }
        # Fill apt_data with basic_info
        for info in basic_info:
            for key, mapping in basic_info_key_mapping.items():
                if mapping in info.lower():
                    apt_data[key] = re.sub(r'\D', '', info)

        print(apt_data)
        
    except Exception as e:
        print(e)

    finally:
        driver.quit()
        return apt_data



link = 'https://loft.com.br/imovel/apartamento-rua-barao-de-itambi-botafogo-rio-de-janeiro-2-quartos-84m2/qtihoc'
apt_data = get_apt_data(link)
print(apt_data)


# apts_data = []
# for link in links:
#     apt_data = get_apt_data(link)
#     apts_data.append(apt_data)
# print(apts_data)

['Valor à vista', 'R$ 750.000', 'Entenda este valor', 'Solicitar visita', 'Tirar dúvidas', 'Agende sua visita gratuita e sem compromissos', 'Parcelas a partir de', 'R$ 6.168/mês', 'Condomínio', 'Isento', 'IPTU', 'R$ 1.623/ano']
price Valor à vista 0
cond. Condomínio 8
iptu IPTU 10
{'price': None, 'cond.': '', 'iptu': '1623', 'area': '84', 'bedrooms': '2', 'suites': '', 'garage': '1', 'bathrooms': '1', 'subway_distance': '347'}
{'price': None, 'cond.': '', 'iptu': '1623', 'area': '84', 'bedrooms': '2', 'suites': '', 'garage': '1', 'bathrooms': '1', 'subway_distance': '347'}
